# *Using ResNet50 pretrained model to classify Concrete Crack images*


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3> 

1.  <a href="https://#item31">Import Libraries and Packages</a>
3.  <a href="https://#item32">Define Global Constants</a>
4.  <a href="https://#item33">Construct ImageDataGenerator Instances</a>
5.  <a href="https://#item34">Compile and Fit Model</a>

</font>

</div>


<a id='item31'></a>


## Import Libraries and Packages


In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

<a id='item32'></a>


## Define Global Constants


Here, we will define constants that we will be using throughout the rest of the lab.

1.  We are obviously dealing with two classes, so *num_classes* is 2.
2.  The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3.  We will training and validating the model using batches of 100 images.


In [4]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>


## Construct ImageDataGenerator Instances


In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.


In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the *flow_from_directory* method to get the training and validation images as follows:


In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_2/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [7]:
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_2/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 10001 images belonging to 2 classes.


Double-click **here** for the solution.

<!-- The correct answer is:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')
-->


<a id='item35'></a>


## Build, Compile and Fit Model


In this section, we will start building our model. We will use the Sequential model class from Keras.


In [8]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.


In [9]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.


In [10]:
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.layers

In [12]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.


In [13]:
model.layers[0].trainable = False

And now using the *summary* attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Next we compile our model using the **adam** optimizer.


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:


In [16]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.


In [17]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/var/folders/ks/m0btnx4d7j38phjw29dfybvw0000gn/T/ipykernel_2094/3326332652.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(
2022-08-14 22:33:10.827105: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2
301/301 [==============================] - 1244s 4s/step - loss: 0.0244 - accuracy: 0.9916 - val_loss: 0.0063 - val_accuracy: 0.9985
Epoch 2/2
301/301 [==============================] - 1261s 4s/step - loss: 0.0051 - accuracy: 0.9990 - val_loss: 0.0044 - val_accuracy: 0.9988


In [18]:
model.save('classifier_resnet_model.h5')

/Users/abdelrahmanibrahim/miniforge3/envs/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
